In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/vosk-model-small-en-us-0-15/vosk-model-small-en-us-0.15/README
/kaggle/input/vosk-model-small-en-us-0-15/vosk-model-small-en-us-0.15/ivector/final.mat
/kaggle/input/vosk-model-small-en-us-0-15/vosk-model-small-en-us-0.15/ivector/online_cmvn.conf
/kaggle/input/vosk-model-small-en-us-0-15/vosk-model-small-en-us-0.15/ivector/splice.conf
/kaggle/input/vosk-model-small-en-us-0-15/vosk-model-small-en-us-0.15/ivector/final.dubm
/kaggle/input/vosk-model-small-en-us-0-15/vosk-model-small-en-us-0.15/ivector/final.ie
/kaggle/input/vosk-model-small-en-us-0-15/vosk-model-small-en-us-0.15/ivector/global_cmvn.stats
/kaggle/input/vosk-model-small-en-us-0-15/vosk-model-small-en-us-0.15/conf/model.conf
/kaggle/input/vosk-model-small-en-us-0-15/vosk-model-small-en-us-0.15/conf/mfcc.conf
/kaggle/input/vosk-model-small-en-us-0-15/vosk-model-small-en-us-0.15/graph/HCLr.fst
/kaggle/input/vosk-model-small-en-us-0-15/vosk-model-small-en-us-0.15/graph/Gr.fst
/kaggle/input/vosk-model-small-en-us-0-

# ***Whisper***

In [2]:
!pip install openai-whisper pandas jiwer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 12.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.1/253.1 MB 6.5 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20240930-py3-none-any.whl size=803405 sha256=d90675b5094fd624244689ee5a261203a0c47568312a586aded8d0c3d1f9e504
  Stored in directory: /root/.cache/pip/wheels/dd/4a/1f/d1c4bf3b9133c8168fe617ed979cab7b14fe381d059ffb9d83
Successfully built openai-whisper
  Attempting uninstall: click
    Found existing installation: click 8.1.7
    Uninstalling click-8.1.7:
      Successfully uninstalled click-8.1.7


In [3]:
import whisper
import os
import pandas as pd
from jiwer import wer

In [4]:
AUDIO_PATH = "/kaggle/input/english-audio-dataset-with-transcription/AI/Dataset"
CSV_PATH = "/kaggle/input/english-audio-dataset-with-transcription/AI/transcription.csv"

In [5]:
df = pd.read_csv(CSV_PATH, delimiter="|", names=["filename", "transcription"])  
df = df.set_index("filename")

In [6]:
whisper_model = whisper.load_model("small")

100%|███████████████████████████████████████| 461M/461M [00:07<00:00, 65.0MiB/s]
/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

In [7]:
AUDIO_PATH = "/kaggle/input/english-audio-dataset-with-transcription/AI/Dataset"
OUTPUT_CSV = "/kaggle/working/whisper_transcriptions.csv"
transcriptions = []  # Store results

# 🔹 Get total file count for progress tracking
total_files = len([f for f in os.listdir(AUDIO_PATH) if f.endswith(".wav")])
processed_count = 0  # Track progress

print(f"🔄 Processing {total_files} audio files with Whisper...")

# 🔹 Loop through all WAV files in the directory
for file in os.listdir(AUDIO_PATH):
    if file.endswith(".wav"):  # Process only WAV files
        file_path = os.path.join(AUDIO_PATH, file)

        # 🔹 Transcribe the audio file
        result = whisper_model.transcribe(file_path)
        transcribed_text = result["text"].strip()  # Clean up whitespace

        # Store result as a dictionary
        transcriptions.append({"filename": file, "whisper_transcription": transcribed_text})

        # 🔹 Progress Tracking
        processed_count += 1
        if processed_count % 10 == 0:  # Print progress every 10 files
            print(f"✅ Processed {processed_count}/{total_files} files...")

# Convert to DataFrame
whisper_df = pd.DataFrame(transcriptions)

# 🔹 Save results to CSV
whisper_df.to_csv(OUTPUT_CSV, index=False)

print(f"✅ Transcriptions saved to {OUTPUT_CSV}")
print(f"🎉 Done! Processed {processed_count} files.")


🔄 Processing 523 audio files with Whisper...
✅ Processed 10/523 files...
✅ Processed 20/523 files...
✅ Processed 30/523 files...
✅ Processed 40/523 files...
✅ Processed 50/523 files...
✅ Processed 60/523 files...
✅ Processed 70/523 files...
✅ Processed 80/523 files...
✅ Processed 90/523 files...
✅ Processed 100/523 files...
✅ Processed 110/523 files...
✅ Processed 120/523 files...
✅ Processed 130/523 files...
✅ Processed 140/523 files...
✅ Processed 150/523 files...
✅ Processed 160/523 files...
✅ Processed 170/523 files...
✅ Processed 180/523 files...
✅ Processed 190/523 files...
✅ Processed 200/523 files...
✅ Processed 210/523 files...
✅ Processed 220/523 files...
✅ Processed 230/523 files...
✅ Processed 240/523 files...
✅ Processed 250/523 files...
✅ Processed 260/523 files...
✅ Processed 270/523 files...
✅ Processed 280/523 files...
✅ Processed 290/523 files...
✅ Processed 300/523 files...
✅ Processed 310/523 files...
✅ Processed 320/523 files...
✅ Processed 330/523 files...
✅ Proce

In [8]:
import pandas as pd
import jiwer  # Library for computing WER

# Load CSV files with correct separators
ground_truth_df = pd.read_csv("/kaggle/input/english-audio-dataset-with-transcription/AI/transcription.csv", 
                              sep="|", header=None, names=["filename", "ground_truth_transcription"])
whisper_df = pd.read_csv("/kaggle/working/whisper_transcriptions.csv")

# Standardize filenames: strip spaces, convert to lowercase
ground_truth_df["filename"] = ground_truth_df["filename"].astype(str).str.strip().str.lower()
whisper_df["filename"] = whisper_df["filename"].astype(str).str.strip().str.lower()

# 🔥 Remove ".wav" from whisper filenames
whisper_df["filename"] = whisper_df["filename"].str.replace(".wav", "", regex=False)

# 🔍 Debugging: Print sample filenames after processing
print("Ground Truth Filenames Sample:", ground_truth_df["filename"].head(10).tolist())
print("Processed Whisper Filenames Sample:", whisper_df["filename"].head(10).tolist())

# Sort both dataframes by filename
ground_truth_df = ground_truth_df.sort_values(by="filename").reset_index(drop=True)
whisper_df = whisper_df.sort_values(by="filename").reset_index(drop=True)

# Merge dataframes on filenames (should work now)
merged_df = pd.merge(ground_truth_df, whisper_df, on="filename", how="inner")

# 🔍 Check if merge is successful
print(f"Merged Dataframe Shape: {merged_df.shape}")

# Drop rows with missing values in transcriptions
merged_df = merged_df.dropna(subset=["ground_truth_transcription", "whisper_transcription"])

# Function to compute WER safely
def calculate_wer(row):
    try:
        return jiwer.wer(str(row["ground_truth_transcription"]), str(row["whisper_transcription"]))
    except Exception as e:
        print(f"Error processing {row['filename']}: {e}")
        return None

# Compute WER
merged_df["WER"] = merged_df.apply(calculate_wer, axis=1)

# Display results
print(merged_df[["filename", "WER"]].head())

# Save final CSV for analysis
merged_df.to_csv("/kaggle/working/evaluation_results.csv", index=False)


Ground Truth Filenames Sample: ['lj001-0001', 'lj001-0002', 'lj001-0003', 'lj001-0004', 'lj001-0005', 'lj001-0006', 'lj001-0007', 'lj001-0008', 'lj001-0009', 'lj001-0010']
Processed Whisper Filenames Sample: ['lj001-0116', 'lj001-0063', 'lj001-0114', 'lj001-0107', 'lj001-0172', 'lj001-0101', 'lj002-0186', 'lj001-0180', 'lj001-0001', 'lj001-0163']
Merged Dataframe Shape: (523, 3)
     filename       WER
0  lj001-0001  0.074074
1  lj001-0002  0.000000
2  lj001-0003  0.083333
3  lj001-0004  0.000000
4  lj001-0005  0.080000


In [9]:
# Compute the overall average WER
overall_wer = merged_df["WER"].mean()

print(f"Overall WER: {overall_wer:.2%}")  # Display as percentage

Overall WER: 15.57%


# ***Faster Whisper***

In [10]:
!pip install faster-whisper
from faster_whisper import WhisperModel  

# Load Faster Whisper with CPU-only mode (lower memory usage)
faster_whisper_model = WhisperModel("small", device="cpu", compute_type="int8")  
 # Use float16 for speed

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.8/38.8 MB 45.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.4/38.4 MB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 89.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 4.7 MB/s eta 0:00:00


vocabulary.txt:   0%|          | 0.00/460k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.37k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.20M [00:00<?, ?B/s]

model.bin:   0%|          | 0.00/484M [00:00<?, ?B/s]

In [11]:
!pip install jiwer
import os
import pandas as pd
from jiwer import wer

In [12]:
AUDIO_PATH = "/kaggle/input/english-audio-dataset-with-transcription/AI/Dataset"
CSV_PATH = "/kaggle/input/english-audio-dataset-with-transcription/AI/transcription.csv"

In [13]:
df = pd.read_csv(CSV_PATH, delimiter="|", names=["filename", "transcription"])  
df = df.set_index("filename")

In [14]:
OUTPUT_CSV = "/kaggle/working/faster_whisper_transcriptions.csv"
transcriptions = []  # Store results

# 🔹 Loop through all WAV files in the directory
total_files = len([f for f in os.listdir(AUDIO_PATH) if f.endswith(".wav")])
processed_count = 0  # Track progress

print(f"🔄 Processing {total_files} audio files...")

for file in os.listdir(AUDIO_PATH):
    if file.endswith(".wav"):  # Process only WAV files
        file_path = os.path.join(AUDIO_PATH, file)

        # 🔹 Transcribe the audio file using Faster Whisper
        segments, _ = faster_whisper_model.transcribe(file_path)
        transcribed_text = " ".join(segment.text for segment in segments)  # Combine segments

        # Store result as a dictionary
        transcriptions.append({"filename": file, "whisper_transcription": transcribed_text})

        processed_count += 1
        if processed_count % 10 == 0:  # Print progress every 10 files
            print(f"✅ Processed {processed_count}/{total_files} files...")

# Convert to DataFrame
faster_whisper_df = pd.DataFrame(transcriptions)

# 🔹 Save results to CSV
faster_whisper_df.to_csv(OUTPUT_CSV, index=False)
print(f"✅ Transcriptions saved to {OUTPUT_CSV}")
print(f"🎉 Done! Processed {processed_count} files.")


🔄 Processing 523 audio files...
✅ Processed 10/523 files...
✅ Processed 20/523 files...
✅ Processed 30/523 files...
✅ Processed 40/523 files...
✅ Processed 50/523 files...
✅ Processed 60/523 files...
✅ Processed 70/523 files...
✅ Processed 80/523 files...
✅ Processed 90/523 files...
✅ Processed 100/523 files...
✅ Processed 110/523 files...
✅ Processed 120/523 files...
✅ Processed 130/523 files...
✅ Processed 140/523 files...
✅ Processed 150/523 files...
✅ Processed 160/523 files...
✅ Processed 170/523 files...
✅ Processed 180/523 files...
✅ Processed 190/523 files...
✅ Processed 200/523 files...
✅ Processed 210/523 files...
✅ Processed 220/523 files...
✅ Processed 230/523 files...
✅ Processed 240/523 files...
✅ Processed 250/523 files...
✅ Processed 260/523 files...
✅ Processed 270/523 files...
✅ Processed 280/523 files...
✅ Processed 290/523 files...
✅ Processed 300/523 files...
✅ Processed 310/523 files...
✅ Processed 320/523 files...
✅ Processed 330/523 files...
✅ Processed 340/523 

In [15]:
import pandas as pd
import jiwer
ground_truth_df = pd.read_csv("/kaggle/input/english-audio-dataset-with-transcription/AI/transcription.csv", 
                              sep="|", header=None, names=["filename", "ground_truth_transcription"])
faster_whisper_df = pd.read_csv(OUTPUT_CSV)

# 🔹 Standardize filenames: strip spaces, convert to lowercase
ground_truth_df["filename"] = ground_truth_df["filename"].astype(str).str.strip().str.lower()
faster_whisper_df["filename"] = faster_whisper_df["filename"].astype(str).str.strip().str.lower()

# 🔹 Remove ".wav" from Faster Whisper filenames
faster_whisper_df["filename"] = faster_whisper_df["filename"].str.replace(".wav", "", regex=False)

# 🔍 Debugging: Print sample filenames
print("Ground Truth Filenames Sample:", ground_truth_df["filename"].head(10).tolist())
print("Processed Faster Whisper Filenames Sample:", faster_whisper_df["filename"].head(10).tolist())

# 🔹 Sort both dataframes by filename
ground_truth_df = ground_truth_df.sort_values(by="filename").reset_index(drop=True)
faster_whisper_df = faster_whisper_df.sort_values(by="filename").reset_index(drop=True)

# 🔹 Merge DataFrames on filename
merged_df = pd.merge(ground_truth_df, faster_whisper_df, on="filename", how="inner")

# 🔍 Check if merge is successful
print(f"Merged Dataframe Shape: {merged_df.shape}")

# 🔹 Drop rows with missing values in transcriptions
merged_df = merged_df.dropna(subset=["ground_truth_transcription", "whisper_transcription"])

# Function to compute WER safely
def calculate_wer(row):
    try:
        return jiwer.wer(str(row["ground_truth_transcription"]), str(row["whisper_transcription"]))
    except Exception as e:
        print(f"Error processing {row['filename']}: {e}")
        return None

# 🔹 Compute WER for Faster Whisper
merged_df["WER"] = merged_df.apply(calculate_wer, axis=1)

# 🔹 Save final evaluation results
merged_df.to_csv("/kaggle/working/faster_whisper_evaluation_results.csv", index=False)

Ground Truth Filenames Sample: ['lj001-0001', 'lj001-0002', 'lj001-0003', 'lj001-0004', 'lj001-0005', 'lj001-0006', 'lj001-0007', 'lj001-0008', 'lj001-0009', 'lj001-0010']
Processed Faster Whisper Filenames Sample: ['lj001-0116', 'lj001-0063', 'lj001-0114', 'lj001-0107', 'lj001-0172', 'lj001-0101', 'lj002-0186', 'lj001-0180', 'lj001-0001', 'lj001-0163']
Merged Dataframe Shape: (523, 3)


In [16]:
overall_wer = merged_df["WER"].mean()
print(f"Overall WER for Faster Whisper: {overall_wer:.2%}")  # Display as percentage

Overall WER for Faster Whisper: 15.63%


# ***Vosk***

In [17]:
!pip install vosk

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 53.7 MB/s eta 0:00:00
  Created wheel for srt: filename=srt-3.5.3-py3-none-any.whl size=22428 sha256=6dfbc6a93c78b8be4548c4c0d979f4545e442e2d0a56b0ed76767958f348087c
  Stored in directory: /root/.cache/pip/wheels/d7/31/a1/18e1e7e8bfdafd19e6803d7eb919b563dd11de380e4304e332
Successfully built srt


In [18]:
import os
import json
import pandas as pd
import jiwer  # Library for computing WER
import wave
import vosk

# 🔹 Load the Vosk model (Make sure you've downloaded a Vosk model and set the correct path)
VOSK_MODEL_PATH = "/kaggle/input/vosk-model-small-en-us-0-15/vosk-model-small-en-us-0.15"
vosk_model = vosk.Model(VOSK_MODEL_PATH)

In [19]:
AUDIO_PATH = "/kaggle/input/english-audio-dataset-with-transcription/AI/Dataset"
OUTPUT_CSV = "/kaggle/working/vosk_transcriptions.csv"
transcriptions = []  # Store results

# 🔹 Get total file count for progress tracking
total_files = len([f for f in os.listdir(AUDIO_PATH) if f.endswith(".wav")])
processed_count = 0  # Track progress

print(f"🔄 Processing {total_files} audio files with Vosk...")

# 🔹 Loop through all WAV files in the directory
for file in os.listdir(AUDIO_PATH):
    if file.endswith(".wav"):  # Process only WAV files
        file_path = os.path.join(AUDIO_PATH, file)

        # 🔹 Open the WAV file
        with wave.open(file_path, "rb") as wf:
            recognizer = vosk.KaldiRecognizer(vosk_model, wf.getframerate())

            transcribed_text = ""
            while True:
                data = wf.readframes(4000)  # Read 4000 frames at a time
                if len(data) == 0:
                    break
                if recognizer.AcceptWaveform(data):
                    result = json.loads(recognizer.Result())
                    transcribed_text += result.get("text", "") + " "

            # Get final result
            final_result = json.loads(recognizer.FinalResult())
            transcribed_text += final_result.get("text", "")

        # Store result as a dictionary
        transcriptions.append({"filename": file, "vosk_transcription": transcribed_text.strip()})

        # 🔹 Progress Tracking
        processed_count += 1
        if processed_count % 10 == 0:  # Print progress every 10 files
            print(f"✅ Processed {processed_count}/{total_files} files...")

# Convert to DataFrame
vosk_df = pd.DataFrame(transcriptions)

# 🔹 Save results to CSV
vosk_df.to_csv(OUTPUT_CSV, index=False)
print(f"✅ Transcriptions saved to {OUTPUT_CSV}")
print(f"🎉 Done! Processed {processed_count} files.")

🔄 Processing 523 audio files with Vosk...
✅ Processed 10/523 files...
✅ Processed 20/523 files...
✅ Processed 30/523 files...
✅ Processed 40/523 files...
✅ Processed 50/523 files...
✅ Processed 60/523 files...
✅ Processed 70/523 files...
✅ Processed 80/523 files...
✅ Processed 90/523 files...
✅ Processed 100/523 files...
✅ Processed 110/523 files...
✅ Processed 120/523 files...
✅ Processed 130/523 files...
✅ Processed 140/523 files...
✅ Processed 150/523 files...
✅ Processed 160/523 files...
✅ Processed 170/523 files...
✅ Processed 180/523 files...
✅ Processed 190/523 files...
✅ Processed 200/523 files...
✅ Processed 210/523 files...
✅ Processed 220/523 files...
✅ Processed 230/523 files...
✅ Processed 240/523 files...
✅ Processed 250/523 files...
✅ Processed 260/523 files...
✅ Processed 270/523 files...
✅ Processed 280/523 files...
✅ Processed 290/523 files...
✅ Processed 300/523 files...
✅ Processed 310/523 files...
✅ Processed 320/523 files...
✅ Processed 330/523 files...
✅ Processe

In [20]:
ground_truth_df = pd.read_csv("/kaggle/input/english-audio-dataset-with-transcription/AI/transcription.csv", 
                              sep="|", header=None, names=["filename", "ground_truth_transcription"])
vosk_df = pd.read_csv(OUTPUT_CSV)

# 🔹 Standardize filenames: strip spaces, convert to lowercase
ground_truth_df["filename"] = ground_truth_df["filename"].astype(str).str.strip().str.lower()
vosk_df["filename"] = vosk_df["filename"].astype(str).str.strip().str.lower()

# 🔹 Remove ".wav" from Vosk filenames
vosk_df["filename"] = vosk_df["filename"].str.replace(".wav", "", regex=False)

# 🔍 Debugging: Print sample filenames
print("Ground Truth Filenames Sample:", ground_truth_df["filename"].head(10).tolist())
print("Processed Vosk Filenames Sample:", vosk_df["filename"].head(10).tolist())

# 🔹 Sort both dataframes by filename
ground_truth_df = ground_truth_df.sort_values(by="filename").reset_index(drop=True)
vosk_df = vosk_df.sort_values(by="filename").reset_index(drop=True)

# 🔹 Merge DataFrames on filename
merged_df = pd.merge(ground_truth_df, vosk_df, on="filename", how="inner")

# 🔍 Check if merge is successful
print(f"Merged Dataframe Shape: {merged_df.shape}")

# 🔹 Drop rows with missing values in transcriptions
merged_df = merged_df.dropna(subset=["ground_truth_transcription", "vosk_transcription"])

# Function to compute WER safely
def calculate_wer(row):
    try:
        return jiwer.wer(str(row["ground_truth_transcription"]), str(row["vosk_transcription"]))
    except Exception as e:
        print(f"Error processing {row['filename']}: {e}")
        return None

# 🔹 Compute WER for Vosk
merged_df["WER"] = merged_df.apply(calculate_wer, axis=1)

# 🔹 Save final evaluation results
merged_df.to_csv("/kaggle/working/vosk_evaluation_results.csv", index=False)

Ground Truth Filenames Sample: ['lj001-0001', 'lj001-0002', 'lj001-0003', 'lj001-0004', 'lj001-0005', 'lj001-0006', 'lj001-0007', 'lj001-0008', 'lj001-0009', 'lj001-0010']
Processed Vosk Filenames Sample: ['lj001-0116', 'lj001-0063', 'lj001-0114', 'lj001-0107', 'lj001-0172', 'lj001-0101', 'lj002-0186', 'lj001-0180', 'lj001-0001', 'lj001-0163']
Merged Dataframe Shape: (523, 3)


In [21]:
overall_wer = merged_df["WER"].mean()
print(f"Overall WER for Vosk: {overall_wer:.2%}")  # Display as percentage

Overall WER for Vosk: 29.38%
